In [ ]:
import cv2
import numpy as np
import time
import SerialCom

# Initialize serial communication
serial_com = SerialCom.SerialCom()

def send_command_to_arduino(command):
    """Sends a command to Arduino and waits for 'DONE' response."""
    serial_com.write(command + "\n")
    print(f"📡 Sent to Arduino: {command}")

    while True:
        response = serial_com.recieve().strip()
        if response == "DONE":
            print(f"✅ Arduino completed: {command}")
            break
        time.sleep(0.1)

# Mapping of board positions to servo motor angles (placeholder)
servo_angles = [
    (0, 0), (0, 90), (0, 180),
    (90, 0), (90, 90), (90, 180),
    (180, 0), (180, 90), (180, 180)
]

board = [' '] * 9

win_combos = [
    [0, 1, 2], [3, 4, 5], [6, 7, 8],
    [0, 3, 6], [1, 4, 7], [2, 5, 8],
    [0, 4, 8], [2, 4, 6]
]

def is_winner(brd, player):
    return any(all(brd[i] == player for i in combo) for combo in win_combos)

def get_available_moves(brd):
    return [i for i, cell in enumerate(brd) if cell == ' ']

def minimax(brd, is_maximizing):
    if is_winner(brd, 'O'):
        return 1
    elif is_winner(brd, 'X'):
        return -1
    elif ' ' not in brd:
        return 0

    scores = []
    for move in get_available_moves(brd):
        brd[move] = 'O' if is_maximizing else 'X'
        score = minimax(brd, not is_maximizing)
        brd[move] = ' '
        scores.append(score)

    return max(scores) if is_maximizing else min(scores)

def best_move(brd):
    best_score = -float('inf')
    move = None
    for i in get_available_moves(brd):
        brd[i] = 'O'
        score = minimax(brd, False)
        brd[i] = ' '
        if score > best_score:
            best_score = score
            move = i
    return move

def detect_colored_objects(frame, color_ranges):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    detected_positions = {color: [] for color in color_ranges}

    for color, ranges in color_ranges.items():
        mask = None
        for lower, upper in ranges:
            lower_np = np.array(lower, dtype=np.uint8)
            upper_np = np.array(upper, dtype=np.uint8)
            partial_mask = cv2.inRange(hsv, lower_np, upper_np)
            mask = partial_mask if mask is None else cv2.bitwise_or(mask, partial_mask)

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 500:
                M = cv2.moments(cnt)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                    detected_positions[color].append((cx, cy))
                    cv2.circle(frame, (cx, cy), 10, (0, 255, 0), 2)
                    cv2.putText(frame, color, (cx - 20, cy - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return detected_positions["red"], detected_positions["blue"], frame

def map_position_to_cell(cx, cy, width, height):
    # Safeguard to ensure column and row are within bounds
    col = min(max(cx * 3 // width, 0), 2)
    row = min(max(cy * 3 // height, 0), 2)
    return row * 3 + col

def update_board_with_detection(cap):
    global board

    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame.")
        return frame

    height, width, _ = frame.shape

    # Draw 3x3 Grid
    cell_width = width // 3
    cell_height = height // 3

    for i in range(1, 3):
        cv2.line(frame, (i * cell_width, 0), (i * cell_width, height), (255, 255, 255), 2)
        cv2.line(frame, (0, i * cell_height), (width, i * cell_height), (255, 255, 255), 2)

    for row in range(3):
        for col in range(3):
            index = row * 3 + col
            cx = col * cell_width + cell_width // 2
            cy = row * cell_height + cell_height // 2
            cv2.putText(frame, str(index), (cx - 10, cy + 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    color_ranges = {
        "red": [([0, 120, 70], [10, 255, 255]), ([170, 120, 70], [180, 255, 255])],
        "blue": [([100, 150, 0], [140, 255, 255])]
    }

    red_pos, blue_pos, processed_frame = detect_colored_objects(frame, color_ranges)

    temp_board = [' '] * 9

    for cx, cy in red_pos:
        idx = map_position_to_cell(cx, cy, width, height)
        if 0 <= idx < 9:
            temp_board[idx] = 'X'

    for cx, cy in blue_pos:
        idx = map_position_to_cell(cx, cy, width, height)
        if 0 <= idx < 9 and temp_board[idx] == ' ':
            temp_board[idx] = 'O'

    board[:] = temp_board
    print_board(board)

    cv2.imshow("Tic-Tac-Toe Camera Feed", processed_frame)
    cv2.waitKey(1)

    return processed_frame

def print_board(brd):
    for i in range(3):
        print(brd[i*3], '|', brd[i*3+1], '|', brd[i*3+2])
        if i < 2:
            print('--+---+--')

def play_game():
    cap = cv2.VideoCapture("http://192.168.1.14:4747/video")
    if not cap.isOpened():
        print("Failed to open video stream.")
        return

    while True:
        try:
            update_board_with_detection(cap)
        except Exception as e:
            print(f"⚠️ Error while updating board: {e}")
            break

        if is_winner(board, 'X'):
            print("Human wins!")
            break
        elif is_winner(board, 'O'):
            print("Robot wins!")
            break
        elif ' ' not in board:
            print("It's a draw!")
            break

        move = best_move(board)
        if move is not None:
            if board[move] == ' ':
                board[move] = 'O'
                angles = servo_angles[move]
                command = f"MOVE {angles[0]} {angles[1]}"
                send_command_to_arduino(command)
                print(f"Robot plays at cell {move}, angles: {angles}")
        else:
            print("No moves left.")
            break

        print_board(board)
        
        time.sleep(2)

    cap.release()
    cv2.destroyAllWindows()

play_game()

COM1 - HHD Software Bridged Serial Port (COM1)
COM2 - HHD Software Bridged Serial Port (COM2)


  |   |  
--+---+--
  |   |  
--+---+--
  |   |  
📡 Sent to Arduino: MOVE 0 0
